Exploratory Data Analysis 

Import Packages

In [1]:
from ast import literal_eval
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering, KMeans

Import Data

In [2]:
df = pd.read_excel("albums_final.xlsx")

Clean Data

In [3]:
# drop all rows that don't have genres
df.dropna(subset=["genres"], axis=0, inplace=True)


# convert genres to list
df["genres"] = df["genres"].apply(literal_eval)

Unique Genres
- check how many unique genres there in the data

In [4]:
genres = df["genres"].to_list()
flat_genres = np.hstack(genres)
len(np.unique(flat_genres))

581